# Middleware Cloud Function User-Model

## Import Library

In [ ]:
import numpy as np
import json
import base64
import googleapiclient.discovery

from PIL import Image
from io import BytesIO
from flask import current_app as app

## Inference Function

### Constans Model Location

In [ ]:
PROJECT = 'cosmic-quarter-312712'
MODEL = 'waste'

### Payload Helper

In [ ]:
def _generate_payload(images):
    return {"instances": images}

### Request Helper

In [ ]:
def _get_model_prediction(service, project, model='demo_model',
                          version=None, body=None):
    if body is None:
        raise NotImplementedError(
            f"_get_model_prediction didn't get any payload for model {model}")

    url = f'projects/{project}/models/{model}'
    if version:
        url += f'/versions/{version}'

    response = service.projects().predict(name=url, body=body).execute()
    return response

In [ ]:
def _connect_service():
    kwargs = {'serviceName': 'ml', 'version': 'v1'}
    return googleapiclient.discovery.build(**kwargs)

In [ ]:
def _preprocessing_image(img_64_encode):
  img_64_decode = base64.decodebytes(img_64_encode)
  img = Image.open(BytesIO(base64.b64decode(img_64_encode)))

  # resize
  size = (256,256)
  img = img.resize(size)
  img = np.array(img)
  xs = np.array([ img ])

  return xs.tolist()

In [ ]:
def get_inference_image(request):
    request_json = request.get_json(silent=True)
    image = request_json['image'] # base64

    image = _preprocessing_image(image)

    service = _connect_service()
    project = PROJECT
    model = MODEL
    response = _get_model_prediction(service, project,
                                     model=model,
                                     body=_generate_payload(image))
    return json.dumps(response)

## Testing

In [ ]:
# img = open('cardboard1.jpg', 'rb')
# img_read = img.read()
# img_64_encode = base64.encodebytes(img_read)
# get_inference_image(img_64_encode)

'{"predictions": [{"dense": [0.9994144439697266, 0.00010214486246695742, 0.00012465484905987978, 0.0003468661743681878, 1.2077418432454579e-05]}]}'